In [11]:
import pandas as pd
import re
import numpy as np

from functools import reduce
import nltk

In [12]:
import stop_words

In [13]:
import gensim
from gensim.models.doc2vec import LabeledSentence

In [14]:
import math

In [15]:
import random

## for cleanup

In [16]:
ps = nltk.stem.snowball.SnowballStemmer('english')

In [17]:
sw = stop_words.get_stop_words('english')

In [18]:
regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string
regex.sub('', 'ab3d*E')

'abdE'

In [19]:
def clean_query(tq):
    result = " "
    t = regex.sub(' ',tq)
    for w in t.split():
        if w not in sw:
            result += ps.stem(w.replace(r'[^a-zA-Z]', '')) + " "
    return result

In [20]:
def prepare_query(tq):
    result = " "
    t = regex.sub(' ',tq)
    for w in t.split():
        if w not in sw:
            result += ps.stem(w.replace(r'[^a-zA-Z]', '')) + " "
    return result.split()

## prepare data

In [21]:
class Doc:
    def __init__(self, title, author, text_lines):
        self.title = title
        self.author=author
        self.text = flat(list(map(lambda l: regex.sub(' ',l).split(), text_lines)))

In [22]:
def flat(list_of_list):
    result = []
    for li in list_of_list:
        for el in li:
            result.append(el)
            
    result = list(filter(lambda l:len(line) > 1, result))
    
    return result

In [23]:
def parse_article_lines(lines):
    text_index = 0
    
    l_s_i = 0
    l_e_i = 0 # label_{start-end}_index
    
    for i,l in enumerate(lines):
        if '.T' in l:
            l_s_i = i+1
        
        if '.A' in l:
            l_e_i = i
        
        if '.W' in l:
            text_index = i+1
    
    author = lines[l_e_i+1]
    
    return Doc(lines[l_s_i:l_e_i], author, lines[text_index:len(lines)])

In [24]:
article_start = 0
article_lines = []

docs = {}

lines = []
with open('Documents.csv') as file:
    lines = file.readlines()

for i,line in enumerate(lines):
    if 'Id' in line and len(line) < 12 and article_start != i:
        article_lines = lines[article_start:i]
        article_start = i
        
        q_id = line.split()[1]
        docs[q_id] = parse_article_lines(article_lines)
    

In [25]:
rel_train = pd.read_csv('relevance_train.csv', sep=',', lineterminator='\n')
rel_test = pd.read_csv('relevance_test.csv')
queries = pd.read_csv('queries.csv', sep='|',lineterminator='\n')

In [26]:
def doc2labeled_sentence(doc):
    return LabeledSentence(words=doc.text, tags=doc.title)

In [27]:
ls = list(map(doc2labeled_sentence, docs.values()))

In [28]:
t = list(map(lambda d: d.text, docs.values()))

docs_as_arr = []
for d in t:
    if len(d) > 0:
        docs_as_arr.append(reduce(lambda w1,w2: w1 + " " + w2 ,d))

#text_as_str = reduce(lambda w1,w2 : w1 + " " + w2, t)

## now prepare model

In [101]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [29]:
SIZE = 10

In [30]:
model = gensim.models.Word2Vec(min_count=1,workers=4, hs=1, negative=0)

In [31]:
model.build_vocab(flat(t))

In [80]:
model = gensim.models.Doc2Vec(size=SIZE, min_count=1, workers=4, hs=1, negative=0)

In [81]:
model.build_vocab(ls)

In [102]:
kw = model.wv

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
tv_docs = TfidfVectorizer(max_features=SIZE, ngram_range=(1,3), stop_words='english')

In [92]:
tv_docs.fit(flat(t))

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f8d23dcfc18>>
Traceback (most recent call last):
  File "/home/lyan/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [93]:
tv_queries = TfidfVectorizer(max_features=SIZE, ngram_range=(1,3), stop_words='english')

In [94]:
tv_queries.fit(queries.iloc[:,1].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

## fix that, i suppose should be some library method or somehow fix vocaulary

In [189]:
SIZE = 300

In [190]:
def get_sen_vector(sen):
    v = np.zeros(SIZE)
    for w in sen:
        try:
            v = np.add(v, kw[w])
        except ValueError as e:
            pass
        except KeyError as e:
            pass
        
    #print(v)
    return v

## prepare now pairs doc - request

In [295]:
def get_vector(doc, q):
    q_v = get_sen_vector(q)
    d_v = get_sen_vector(doc)
    return np.hstack((q_v, d_v))

In [296]:
vectors = []
labels = []

for row in rel_train.iterrows():
    qid = row[1][0]
    did = row[1][1]
    rel = row[1][2]
    
    if str(did) in docs and qid in queries['QueryId']:
            labels.append(rel)
            vectors.append(get_vector(docs[str(did)].text, queries[queries['QueryId']==qid]))
        
vectors = np.array(vectors)
labels = np.array(labels).reshape(len(labels),1)

## same for tfidf from sklearn

In [118]:
vectors = []
labels = []

for row in rel_train.iterrows():
    qid = row[1][0]
    did = row[1][1]
    rel = row[1][2]
    
    if str(did) in docs and qid in queries['QueryId']:
            labels.append(rel)
            
            vectors.append(r)
        
vectors = np.array(vectors).reshape(len(vectors), 2 * SIZE)
labels = np.array(labels).reshape(len(labels),1)

NameError: name 'tv_queries' is not defined

In [44]:
from sklearn.preprocessing import MinMaxScaler

In [57]:
mms = MinMaxScaler()

In [58]:
labels_mms = mms.fit_transform(labels)

/home/dev/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


## metrics

In [74]:
import math

In [75]:
def dcg(rels, p):
    s = 0
    for i in range(0,p):
        rel = rels[i]
        s += (rel / math.log(rel, 2))
    return s

In [76]:
def ndcg(ideal, rels, p):
    return dcg(rels, p) / dcg(ideal,p)

## prepare model

In [206]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [207]:
trainX, testX, trainY, testY = train_test_split(vectors, labels)

In [208]:
dtrain = xgb.DMatrix(trainX, trainY)
dtest = xgb.DMatrix(testX, testY)

In [275]:

param = {'silent':1, 'objective':'rank:pairwise' }
param['nthread'] = 8
param['eval_metric'] = 'ndcg'
param['eta'] = 0.075
param['max_depth'] = 5
param['silent'] = 1
param['n_estimators'] = 500
param['early_stopping_rounds'] = 30

watchlist = [ (dtrain,'train'), (dtest, 'test') ]

In [262]:
bst = xgb.train(param, dtrain, 1000, watchlist,verbose_eval=100)

[0]	train-:0.526954	test-:0.448076
[100]	train-:0.503106	test-:0.426402
[200]	train-:0.503364	test-:0.427718
[300]	train-:0.503317	test-:0.425587
[400]	train-:0.503503	test-:0.430346
[500]	train-:0.503231	test-:0.430655
[600]	train-:0.503348	test-:0.428516
[700]	train-:0.50358	test-:0.427002
[800]	train-:0.503348	test-:0.426082
[900]	train-:0.503445	test-:0.424861


In [263]:
pred = bst.predict(dtest)

In [264]:
ndcg(testY, pred, 5)

ValueError: math domain error

## linear model

In [217]:
from sklearn.linear_model import LinearRegression

In [218]:
lr = LinearRegression()
lr.fit(trainX, trainY)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [219]:
pred = lr.predict(testX)

In [220]:
from sklearn.metrics import mean_squared_error

In [221]:
mean_squared_error(pred, testY)

2.3206557810416113

In [222]:
ndcg(testY, pred,5)

array([ 0.04020345])

## rank svm

In [299]:
def f(x):
    X = []
    y = []

    for row in x.iterrows():
        qid = row[1][0]
        did = row[1][1]
        rel = row[1][2]

        if str(did) in docs and qid in queries['QueryId']:
                y.append(rel)
                X.append(get_vector(docs[str(did)].text, queries[queries['QueryId']==qid]))

    X = np.array(X)
    y = np.array(y).reshape(len(y),1)
    
    rsvm.fit(X,y)

In [300]:
rel_train.groupby('QueryId').apply(f)

IndexError: index 1 is out of bounds for axis 1 with size 1

In [278]:
from ranking import RankSVM

In [279]:
rsvm = RankSVM()

In [282]:
rsvm.fit(trainX, trainY)

IndexError: index 1 is out of bounds for axis 1 with size 1

## preparing results

In [169]:
from sklearn.metrics.pairwise import cosine_similarity

In [170]:
def get_text_from_query_by(i):
    text = queries[queries['QueryId']==i].values[0][1]
    return clean_query(text)

In [171]:
def to_nparray(res):
    print(res)
    result = np.zeros((len(res), 2))
    for i,row in enumerate(res):
        result[i,0] = row[1]
        result[i,1] = row[2]
    print(result)
    return result

In [172]:
def rank_block_by_dist(q_ids, d_ids):
    sim = []
    for q_id, d_id in zip(q_ids, d_ids):
        dv = get_sen_vector(docs[str(d_id)].text)
        qv = get_sen_vector(get_text_from_query_by(q_id))
        sim.append(cosine_similarity([dv],[qv]))
        
    return to_nparray(sorted(zip(sim, q_ids, d_ids), key=lambda key: key[0]))

In [199]:
def rank_block_by_pred(q_ids, d_ids):
    result = []
    for q_id, d_id in zip(q_ids, d_ids):
        d_vec = get_sen_vector(docs[str(d_id)].text).reshape(SIZE, 1)
        q_vec = get_sen_vector(get_text_from_query_by(q_id)).reshape(SIZE,1)
        r = np.hstack((q_vec, d_vec)).reshape(1, 2 * SIZE)
        m = xgb.DMatrix(r)
        pred = bst.predict(m)
        result.append(pred)
        
    print(result)
        
    return to_nparray(sorted(zip(result, q_ids, d_ids), key=lambda key: key[0]))

In [174]:
def rank_tfidf(q_ids, d_ids):
    result = []
    for q_id, d_id in zip(q_ids, d_ids):
        q_v = np.mean(tv_queries.transform(queries[queries['QueryId']==q_id]), axis=0)
        d_v = np.mean(tv_docs.transform(docs[str(d_id)].text), axis=0)
        r = np.hstack((q_v, d_v)).reshape(2 * SIZE)
        m = xgb.DMatrix(r)
        pred = bst.predict(m)
        result.append(pred)
            
    return to_nparray(sorted(zip(result, q_ids, d_ids), key=lambda key: key[0], reverse=True))

## xgb test pred

In [265]:
test_vectors = []

for row in rel_test.iterrows():
    qid = row[1][0]
    did = row[1][1]
    
    if str(did) in docs and qid in queries['QueryId']:
        q_v = get_sen_vector(queries[queries['QueryId']==qid])
        d_v = get_sen_vector(docs[str(did)].text)
        t = np.hstack((q_v, d_v))
        test_vectors.append(t)
    else:
        test_vectors.append(np.zeros((SIZE * 2)))
        
test_vectors = np.array(test_vectors)

In [266]:
test_mat = xgb.DMatrix(test_vectors)

In [267]:
predict = bst.predict(test_mat)

In [268]:
subm = rel_test.copy()

In [269]:
subm['score'] = predict

In [270]:
res = subm.groupby('QueryId').apply(lambda x:x.sort_values('score', ascending=False))

In [271]:
res.to_csv('xgb_w2v_rank_ndcg.csv', index=False, columns=('QueryId', 'DocumentId'))